In [20]:
import os
import csv

In [21]:
from supabase import create_client, Client

In [30]:
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")

In [28]:
supabase: Client = create_client(url, key)

In [29]:
data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [32]:
def generate_rounds(rounds_csv, competition_id: int):
    rounds_insert = []
    with open(rounds_csv, mode='r', encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            row['competition_id'] = competition_id
            rounds_insert.append(row)
    return rounds_insert

def add_rounds_to_db(rounds_csv, competition_id, supabase_client):
    supabase.table("competitionround").upsert(generate_rounds(rounds_csv, competition_id), ignore_duplicates=True, on_conflict="round_number, competition_id").execute()

In [ ]:
add_rounds_to_db("rounds-2023-ncaa-tournament.csv", 1, supabase)

In [39]:
def generate_conferences(conferences_csv, league_unique: str):
    conferences_insert = []
    with open(conferences_csv, mode='r', encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            trimmed_row = {key: row[key].lower() if key == 'conference_unique' else row[key] for key in row.keys()
                           & {'conference_unique', 'conference_name'}}
            trimmed_row['league_unique'] = league_unique.lower()
            conferences_insert.append(trimmed_row)
    return conferences_insert

def add_conferences_to_db(conferences_csv, leage_unique: str, supabase_client):
    supabase.table("conference").insert(generate_conferences(conferences_csv, leage_unique)).execute()

In [ ]:
add_conferences_to_db("ncaambb-conferences-2023.csv", "ncaambb", supabase)

In [113]:
from datetime import date
def generate_teams(team_stats_csv, league_unique: str, competition_id: int, expected_num_teams: int, thru=date.today()):
    team_insert = []
    team_stat_insert = []
    with open(team_stats_csv, mode='r', encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            team_unique = row['abbreviation'].lower()
            if team_unique[:-6] == "/women":
                team_unique = team_unique[:-6]
            if team_unique[:-4] == "/men":
                team_unique = team_unique[:-4]
            team_row = {'league_unique': league_unique,
                        'team_unique': team_unique,
                        'team_name': row['name']}
            team_insert.append(team_row)
            additional_team_stats = {key: float(row[key]) if row[key] else 0 for key in row.keys() & {'effective_field_goal_percentage','strength_of_schedule','assist_percentage', 'free_throw_attempt_rate',  'offensive_rating', 'opp_effective_field_goal_percentage', 'two_point_field_goal_percentage', 'three_point_field_goal_percentage', 'pace', 'three_point_attempt_rate', 'true_shooting_percentage', 'turnover_percentage'}}
            team_stat_row = {key: row[key] for key in row.keys()
                             & {'games_played','wins', 'losses','conference_wins', 'conference_losses'}}
            team_stat_row['team_unique'] = team_unique
            team_stat_row['competition_id'] = competition_id
            team_stat_row['league_unique'] = league_unique
            team_stat_row['conference_unique'] = row['conference'].lower()
            team_stat_row['additional_team_stats'] = additional_team_stats
            team_stat_row['thru'] = thru
            team_stat_insert.append(team_stat_row)
    if len(team_insert) != expected_num_teams or len(team_stat_insert) != expected_num_teams:
        raise ValueError("Number of teams in csv does not match expected number of teams")
    else:
        return [team_insert, team_stat_insert]


def add_teams_to_db(teams_csv, leage_unique: str, competition_id: str, expected_num_teams: int, thru, supabase_client):
    [teams, team_stats] = generate_teams(teams_csv, leage_unique, competition_id, expected_num_teams, thru)
    supabase.table("team").upsert(teams, ignore_duplicates=True,
                                  on_conflict="team_unique, league_unique").execute()
    supabase.table("stats_team_precompetitionsnapshot_basketball").upsert(team_stats, ignore_duplicates=False, on_conflict='team_unique,competition_id, league_unique').execute()

In [114]:
add_teams_to_db(teams_csv="2023_acc_tournament_team_stats.csv", leage_unique="ncaambb", competition_id=3, expected_num_teams=15, thru='2023-03-07', supabase_client=supabase)

In [100]:
def generate_team_seeds(team_seeds_csv, league_unique: str, competition_id: int, expected_num_teams: int):
    team_seed_insert = []
    with open(team_seeds_csv, mode='r', encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            team_seed_row = {key: row[key].lower() if key == 'team_unique' else row[key] for key in row.keys()
                             & {'team_unique', 'seed','overall_seed', 'region'}}
            team_seed_row['competition_id'] = competition_id
            team_seed_row['league_unique'] = league_unique
            team_seed_insert.append(team_seed_row)
    if len(team_seed_insert) != expected_num_teams:
        raise ValueError("Number of teams in seed file does not match expected number of teams")
    return team_seed_insert


def add_seeds_to_db(team_seeds_csv, league_unique: str, competition_id: str, expected_num_teams: int, supabase_client):
    seeds = generate_team_seeds(team_seeds_csv, league_unique, competition_id, expected_num_teams)
    supabase_client.table("team_competition").upsert(seeds, ignore_duplicates=False,
                                  on_conflict="team_unique, competition_id, league_unique").execute()

In [101]:
add_seeds_to_db(team_seeds_csv="2023_acc_tournament_team_seeds.csv", league_unique="ncaambb", competition_id=3, expected_num_teams=15, supabase_client=supabase)

In [34]:
from datetime import date
def generate_players(player_stats_csv, competition_id: int, league_unique:str, thru=date.today()):
    player_insert = []
    player_competition_insert= []
    player_stat_insert = []
    with open(player_stats_csv, mode='r', encoding='utf-8-sig') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            additional_player_stats = {key: float(row[key]) if row[key] else 0 for key in row.keys()
                             & {'games_started','minutes_played', 'effective_field_goal_percentage', 'field_goals', 'field_goal_attempts','field_goal_percentage', 'two_pointers', 'two_point_attempts', 'two_point_percentage', 'three_pointers', 'three_point_attempts','three_point_percentage', 'free_throws', 'free_throw_attempts','free_throw_percentage', 'free_throw_attempt_rate', 'offensive_rebounds', 'defensive_rebounds', 'steals', 'turnovers', 'personal_fouls', 'usage_percentage','true_shooting_percentage', 'player_efficiency_rating'}}
            player_unique = row['player_id'].lower()
            team_unique=row['team_abbreviation'].lower()
            player_row = {'player_unique': player_unique,'player_name': row['name']}
            if 'birthdate' in row:
                player_row['birthdate'] = row['birthdate']
            if 'position' in row:
                player_row['position'] = row['position']
            player_stat_row = {key: int(float(row[key])) if row[key] else 0 for key in row.keys()
                             & {'games_played','points', 'assists','blocks' }}
            player_competition_row = {'player_unique': player_unique, 'competition_id': competition_id, 'league_unique': league_unique, 'team_unique': team_unique, 'inactive': False}
            player_stat_row['player_unique'] = player_unique
            player_stat_row['competition_id'] = competition_id
            player_stat_row['rebounds'] = int(float(row['total_rebounds'])) if row['total_rebounds'] else 0
            player_stat_row['thru'] = thru
            player_stat_row['additional_player_stats'] = additional_player_stats
            player_insert.append(player_row)
            player_competition_insert.append(player_competition_row)
            player_stat_insert.append(player_stat_row)
    return [player_insert, player_competition_insert, player_stat_insert]


def add_players_to_db(player_stats_csv,  competition_id: str, league_unique: str, thru, supabase_client):
    [players, player_competition, player_stats] = generate_players(player_stats_csv, competition_id, league_unique, thru)
    supabase.table("player").upsert(players, ignore_duplicates=True,
                                  on_conflict="player_unique").execute()
    supabase.table("player_competition").upsert(player_competition, ignore_duplicates=False, on_conflict='player_unique,competition_id').execute()
    supabase.table("stats_player_precompetitionsnapshot_basketball").upsert(player_stats, ignore_duplicates=False, on_conflict='player_unique,competition_id').execute()

In [108]:
add_players_to_db(player_stats_csv="2023_acc_tournament_player_stats.csv", competition_id=3, league_unique='ncaambb', thru='2023-03-07', supabase_client=supabase)